<a href="https://colab.research.google.com/github/SolNam-UI/CTD/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import 하는 곳

In [ ]:
!pip install quickdraw tensorflow ColabTurtle Pillow

In [ ]:
import numpy as np
import random
from PIL import Image
from quickdraw import QuickDrawDataGroup
from keras import layers
from keras import models
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from ColabTurtle import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/파이썬 프로젝트 CTD"
!git clone https://github.com/SolNam-UI/CTD.git
!git clone https://gist.github.com/8409b3feec20f159d8a50b0a811d3bca.git #Colab 환경에서 사용자가 그릴 수 있게 해주는 오픈소스
%run 8409b3feec20f159d8a50b0a811d3bca/draw.py
%cd "/content/drive/Mydrive/파이썬 프로젝트 CTD/storage"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/파이썬 프로젝트 CTD'
/content
fatal: destination path 'CTD' already exists and is not an empty directory.
fatal: destination path '8409b3feec20f159d8a50b0a811d3bca' already exists and is not an empty directory.
[Errno 2] No such file or directory: '/content/drive/Mydrive/파이썬 프로젝트 CTD/storage'
/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Turtle로 사용자가 그림 그리게 하기 (동렬)

In [ ]:
import turtle
import tkinter
from tkinter import filedialog, messagebox
from PIL import Image # cmd에서 (pip install Pillow) 을 통해 라이브러리 설치 후 사용 가능
import os

def turtle_mouse_drawing():

    window_for_turtle_drawing = tkinter.Tk()
    window_for_turtle_drawing.title("터틀을 활용한 마우스로 그림 그리기 창")
    window_for_turtle_drawing.geometry("1000x800")

    canvas_for_drawing = tkinter.Canvas(window_for_turtle_drawing, width=1000, height=800, bg="white")
    canvas_for_drawing.pack() #윈도우 창에 맞게 canvas 생성

    screen_for_turtle = turtle.TurtleScreen(canvas_for_drawing)
    screen_for_turtle.tracer(0) #터틀 스크린 생성

    drawing_turtle = turtle.RawTurtle(screen_for_turtle)
    drawing_turtle.speed(0)
    drawing_turtle.penup()

    drawing = False #마우스 드래그 여부 나타내기

    def start_drawing_with_turtle(event):

        nonlocal drawing
        drawing = True
        drawing_turtle.penup()
        drawing_turtle.goto(event.x - 500, 400 - event.y) #tkinterface 내의 좌표를 Turtle 좌표로 변환하기
        drawing_turtle.pendown()

    def draw_change(event):

        if drawing:

            drawing_turtle.goto(event.x - 500, 400 - event.y)
            screen_for_turtle.update()

    def stop_drawing(event):

        nonlocal drawing
        drawing = False
        drawing_turtle.penup()

    def save_turtle_drawing_as_gif_file():

        file_path = filedialog.asksaveasfilename(

            default_extension=".gif", # 기본 파일을 .gif 형식으로 지정
            file_type= [("GIF files", "*.gif"), ("All files", "*.*")],
            title="그림을 GIF 파일로 저장"

        )

        if file_path:

            try:

                try_ps_file = "try_drawing.ps"
                canvas_for_drawing.postscript(file = try_ps_file, colormode='color')

                done_image = Image.open(try_ps_file)
                done_image.save(file_path, "gif")

                os.remove(try_ps_file)

                massagebox.showinfo("저장이 완료되었습니다", f"해당 그림이 '{file_path}' 에 GIF 형식의 파일로 저장되었습니다.")

            except Exception as error:

                messagebox.showerror("저장 중에 오류가 발생했습니다", f"그림을 저장하는 중에 오류가 발생했습니다 : {error}")




    canvas_for_drawing.bind("<Button-1>", start_drawing_with_turtle)
    canvas_for_drawing.bind("<B1-Motion>", draw_change)
    canvas_for_drawing.bind("<ButtonRelease-1>", stop_drawing)

    save_button = tkinter.Button(window_for_turtle_drawing, text="GIF 형식의 파일로 저장", command = save_turtle_drawing_as_gif_file)

    save_button.pack(pady=10)


    window_for_turtle_drawing.mainloop()

if __name__ == "__main__":
    draw(filename="my_drawing.png", w=400, h=400, line_width=15)

    # 나중에 수정해두기


KeyboardInterrupt: 

CNN 구현 및 이미지 학습 (윤성)

In [ ]:
# 모델 정의
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation = "relu", input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(units = 64, activation = "relu"))
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

# 학습할 이미지 리스트
trainList = ["cat", "dog", "bird", "mouse"]

# 이미지 학습
trainImg, trainLabel = [], []
for i, label in enumerate(trainList):
    drawings = list(QuickDrawDataGroup(label).drawings)[:100]
    for d in drawings:
        img = d.image.resize((28, 28)).convert("L")
        img_array = np.array(img) / 255.0
        trainImg.append(img_array)
        trainLabel.append(i)

trainImg = np.array(trainImg)
trainImg = trainImg.reshape(-1, 28, 28, 1)

trainLabel = np.array(trainLabel)

# 이미지 전처리
img = Image.open(imgPath).resize((28, 28)).convert("L")
imgTemp = np.array(img) / 255.0
inData = imgTemp.reshape(1, 28, 28, 1)

# 모델 학습
model.fit(trainImg, trainLabel, epochs=2, batch_size=32)

# 예측
prediction = model.predict(inData)

print("정답은: ", trainList[np.argmax(prediction)] + "입니다.")

loading cat drawings
load complete
loading dog drawings
load complete
loading bird drawings
load complete
loading mouse drawings
load complete
Epoch 1/2
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.2119 - loss: 1.9776
Epoch 2/2
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.2675 - loss: 1.3863
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
정답은:  bird입니다.


Turtle로 출력하기 (동렬)

ModuleNotFoundError: No module named 'turtle2img'